## Using interactive parameters with maps

Using `wf.parameter`, you can add "placeholders" in your code, which you can adjust interactively when you `visualize` an Imge on the map.

For example, you could add a parameter for a threshold value, or a date range, then control those using a widget.

In [18]:
import descarteslabs.workflows as wf
import ipywidgets
import datetime

In [4]:
wf.map.zoom = 9
wf.map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart Jupyter and re-run this notebook.


We're making a mean composite of the VV band in Sentinel-1, then masking out values below a certain threshold, which we pick interactively.

In [23]:
s1_vv = (
    wf.ImageCollection
        .from_id("sentinel-1:GRD", start_datetime="2019-08-01", end_datetime="2019-10-01")
        .mean(axis="images")
        .pick_bands("vv")
)

threshold_param = wf.parameter('threshold', wf.Float)
# ^ make a parameter named 'threshold' of type Float

masked_vv = s1_vv.mask(s1_vv < threshold_param)

masked_vv.visualize(
    "Masked VV",
    colormap="viridis",
    scales=[0, 0.5],
    threshold=0.04
    # ^ pass in an intial value for the "threshold" parameter.
    # Any extra keyword arguments to `visualze` are interpreted as parameters.
    # The names must match up with parameter names you've created with `wf.parameter`.
)

Changing the `threshold` input above will update the map.

Since you passed in `0.5` for `threshold`, Workflows inferred that the parameter was a float,
and by default displays a text box where you can enter floats.

But you may not just want a text box. For more customization, you can also pass in `ipywidgets` Widgets as parameters—they'll be used as the controls.

-------

Here, we'll do a similar computation, but add parameters to pick the start date, time window duration, and pass direction.

In [24]:
start = wf.parameter("start", wf.Datetime)
end = start + wf.Timedelta(weeks=wf.parameter("weeks", wf.Int))

s1 = wf.ImageCollection.from_id("sentinel-1:GRD", start_datetime=start, end_datetime=end)

pass_direction = wf.parameter("pass_dir", wf.Str)

s1_mean = s1.filter(lambda img: img.properties['pass'] == pass_direction).mean(axis="images")
vv = s1_mean.pick_bands("vv")

masked_s1 = vv.mask(vv < wf.parameter('threshold', wf.Float))

In [25]:
lyr = masked_s1.visualize(
    "Masked VV",
    colormap="magma",
    scales=[0, 0.5],
    
    start=ipywidgets.DatePicker(value=datetime.date(2019, 8, 1)),
    weeks=ipywidgets.IntSlider(min=0, max=12, value=4),
    pass_dir=ipywidgets.Dropdown(
        options=["ASCENDING", "DESCENDING"]
    ),
    threshold=ipywidgets.FloatSlider(
        min=0, max=1, step=0.01
    ),
)
lyr

You can also adjust parameters on a layer programatically by assigning to `lyr.parameters.<param-name>`:

In [26]:
lyr.parameters.threshold = 0.06
lyr.parameters.weeks = 3
lyr.parameters.pass_dir = "DESCENDING"